In [233]:
import pandas as pd
import numpy as np

In [236]:
file_name = 'RfqData.xlsx'
sheettr =  "Training RFQs"
sheettst= "OOS RFQs"


training = pd.read_excel(io=file_name, sheet_name=sheettr)
test = pd.read_excel(io=file_name, sheet_name=sheettst)
print(training.head(5))  # print first 5 rows of the dataframe

      Time    Bond   Side    Notional N_group Counterparty  MidPrice  \
0  25000.0  Bond_2  Offer  10000000.0     big       Ctpy_0    124.01   
1  25001.0  Bond_0    Bid      1000.0   small       Ctpy_1     98.07   
2  25002.0  Bond_1  Offer      1000.0   small       Ctpy_1    170.30   
3  25003.0  Bond_0    Bid     20000.0     big       Ctpy_0     98.00   
4  25004.0  Bond_1  Offer      1000.0   small       Ctpy_3    171.12   

   QuotedPrice  Delta  Competitors  Profit  Status  Adjusted profit  Traded  \
0       124.25   0.24          1.0   -0.01     0.0             0.00  MISSED   
1        98.06   0.01          1.0    0.02     1.0             0.02    DONE   
2       170.40   0.10          1.0    0.24     0.0             0.00  MISSED   
3        97.98   0.02          4.0   -0.04     1.0            -0.04    DONE   
4       171.16   0.04          2.0    0.30     1.0             0.30    DONE   

   NextMidPrice  
0        124.24  
1         98.08  
2        170.64  
3         97.94  
4 

In [445]:
def profitdist(trades):
    delta_min = min(list(trades.Delta))
    delta_max = max(list(trades.Delta))
    trades_total = len(list(trades.Delta))
    delta_performance = []
    delta_curr = round(delta_min,2)
    while delta_curr<=delta_max:
        passing_trades = trades[trades.Delta>=delta_curr]
        passing_trades_n = len(list(passing_trades.Delta))
        done_trades = trades[(trades.Delta>=delta_curr)&(trades.Status==1)]
        done_trades_n = len(list(done_trades.Delta))
        profit = round(sum(list(done_trades.Profit)),2)
        if done_trades_n>1:
            profit_var = round(np.var(list(done_trades.Profit)),5)
            profit = profit/done_trades_n
        else:
            profit_var = 0
        delta_performance.append((delta_curr,trades_total,passing_trades_n,done_trades_n,profit,profit_var))
        delta_curr=round(delta_curr+0.01,2)
        
    return delta_performance

def best_perf(delta_performance):
    best_coeff = 0
    best_perf = "NoPositiveTrades"
    for perf in delta_performance:
        coeff = 0
        if perf[3]>10:
            coeff = perf[4]/perf[5]
        if coeff>best_coeff:
            best_coeff = coeff
            best_perf = perf
    return best_perf

def partition(data):
    bond_types = set(data["Bond"])
    bond_types.add('all')
    bond_types = set(filter(lambda x: x == x , bond_types))
    side_types = set(data["Side"])
    side_types.add('all')
    side_types = set(filter(lambda x: x == x , side_types))
    notional_types = set(data["N_group"])
    notional_types.add('all')
    notional_types = set(filter(lambda x: x == x , notional_types))
    cp_types = set(data["Counterparty"])
    cp_types.add('all')
    cp_types = set(filter(lambda x: x == x , cp_types))
    cmp_types = set(data["Competitors"])
    cmp_types.add('all')
    cmp_types = set(filter(lambda x: x == x , cmp_types))
    
    results = []
    for b_type in bond_types:
       #print(b_type)
        for s_type in side_types:
            #print(s_type)
            for n_type in notional_types:
                #print(n_type)
                for cp_type in cp_types:
                    #print(cp_type)
                    for cmp_type in cmp_types:
                        #print(cmp_type)
                        cut_param = []
                        if b_type != 'all':
                            b_used = [b_type]
                            cut_param.append(b_type)
                        else:
                            b_used = bond_types
                            
                        if s_type != 'all':
                            s_used = [s_type]
                            cut_param.append(s_type)
                        else:
                            s_used = side_types
                            
                        if n_type != 'all':
                            n_used = [n_type]
                            cut_param.append(n_type)
                        else:
                            n_used = notional_types
                            
                        if cp_type != 'all':
                            cp_used = [cp_type]
                            cut_param.append(cp_type)
                        else:
                            cp_used = cp_types
                            
                        if cmp_type != 'all':
                            cmp_used = [cmp_type]
                            cut_param.append(cmp_type)
                        else:
                            cmp_used = cmp_types
                        
                        chosen_data = data[data.Bond.isin(b_used) 
                                               & data.Side.isin(s_used) 
                                               & data.N_group.isin(n_used) 
                                               & data.Counterparty.isin(cp_used) 
                                               & data.Competitors.isin(cmp_used)]
                            
                        if chosen_data.empty:
                            results.append([cut_param,"NoData"])
                        else:
                            perfs = profitdist(chosen_data)
                            best = best_perf(perfs)
                            results.append([cut_param,best])
    
    return results

def anticonditions(conditions, data):
    anti_conds = set()
    bond_types = set(data["Bond"])
    side_types = set(data["Side"])
    notional_types = set(data["N_group"])
    cp_types = set(data["Counterparty"])
    cmp_types = set(data["Competitors"])
    bond_types = set(filter(lambda x: x == x , bond_types))
    side_types = set(filter(lambda x: x == x , side_types))
    notional_types = set(filter(lambda x: x == x , notional_types))
    cp_types = set(filter(lambda x: x == x , cp_types))
    cmp_types = set(filter(lambda x: x == x , cmp_types))
    type_sets = [bond_types,side_types,notional_types,cp_types, cmp_types]
    
    for cond in conditions:
        for type_set in type_sets:
            if cond in type_set:
                local_anti_conds = type_set.copy()
                local_anti_conds.remove(cond)
                anti_conds = anti_conds.union(local_anti_conds)
            
    return anti_conds

def pick_fit_partitions(conditions,full_partition, data):
    anticonds = anticonditions(conditions, data)
    fitting = []
    for cut in full_partition:
        cut_param = cut[0]
        fit = 1
        for cond in anticonds:
            if cond in cut_param:
                fit = 0
        if cut[1]=="Null":
            fit = 0
        if fit == 1:
            fitting.append(cut)
    return fitting

def pick_best_action(conditions,full_partition,data):
    fit_partitions = pick_fit_partitions(conditions,full_partition, data)
    performances = [c[1] for c in fit_partitions if type(c[1]) is tuple]
    best = best_perf(performances)
    full_data = [a for a in fit_partitions if a[1]==best]
    return full_data

In [446]:
perf = profitdist(training)

In [447]:
best_perf(perf)

(0.14, 5989, 596, 23, 0.06304347826086956, 0.0297)

In [448]:
full_partition = partition(training)

In [449]:
full_partition[:10]

[[['Bid', 'Ctpy_1', 1.0], 'NoPositiveTrades'],
 [['Bid', 'Ctpy_1', 2.0], (0.03, 200, 132, 48, 0.02625, 0.01908)],
 [['Bid', 'Ctpy_1', 3.0], (0.05, 176, 75, 12, 0.1708333333333333, 0.02731)],
 [['Bid', 'Ctpy_1', 4.0], (0.02, 181, 123, 65, 0.028307692307692308, 0.02174)],
 [['Bid', 'Ctpy_1'], (0.09, 760, 181, 13, 0.12692307692307692, 0.03179)],
 [['Bid', 'Ctpy_2', 1.0], 'NoPositiveTrades'],
 [['Bid', 'Ctpy_2', 2.0], 'NoPositiveTrades'],
 [['Bid', 'Ctpy_2', 3.0], 'NoPositiveTrades'],
 [['Bid', 'Ctpy_2', 4.0], 'NoPositiveTrades'],
 [['Bid', 'Ctpy_2'], 'NoPositiveTrades']]

In [450]:
[a for a in full_partition if len(a[0])<1]

[[[], (0.14, 5000, 596, 23, 0.06304347826086956, 0.0297)]]

In [451]:
conditions = ["Bid","Ctpy_1",1]
anticonditions(conditions, training)

{2.0, 3.0, 4.0, 'Ctpy_0', 'Ctpy_2', 'Ctpy_3', 'Offer'}

In [452]:
pick_fit_partitions(conditions,full_partition,training)[:5]

[[['Bid', 'Ctpy_1', 1.0], 'NoPositiveTrades'],
 [['Bid', 'Ctpy_1'], (0.09, 760, 181, 13, 0.12692307692307692, 0.03179)],
 [['Bid', 1.0], (0.06, 659, 225, 12, 0.09749999999999999, 0.04175)],
 [['Bid'], (0.13, 2489, 317, 15, 0.10600000000000001, 0.02856)],
 [['Bid', 'small', 'Ctpy_1', 1.0],
  (0.01, 150, 131, 48, 0.004166666666666667, 0.01353)]]

In [453]:
pick_best_action(conditions,full_partition,training)

[[['Bond_0', 'Bid', 'small'],
  (0.04, 883, 276, 33, 0.03333333333333333, 0.00557)]]

In [454]:
test.loc[4]

Time               30004.0
Bond                Bond_0
Side                 Offer
Notional        10000000.0
N_group                big
Counterparty        Ctpy_1
MidPrice              84.6
QuotedPrice          84.64
Competitors            2.0
Traded                DONE
NextMidPrice         84.61
Name: 4, dtype: object

In [455]:
profits = []
for i in range(len(test)):
    tr = test.loc[i]
    params = [tr.Bond,tr.Side,tr.N_group,tr.Counterparty,tr.Competitors]
    #print(params)
    best_action = pick_best_action(params,full_partition,training)[0]
    #print(best_action)
    action = best_action[1][0]
    if tr.Side == "Bid":
        our_q = tr.MidPrice-action
        if tr.QuotedPrice>our_q:
            profit = 0
        else:
            profit = tr.NextMidPrice - our_q
    if tr.Side == "Offer":
        our_q = tr.MidPrice+action
        if tr.QuotedPrice<our_q:
            profit = 0
        else:
            profit = our_q - tr.NextMidPrice
    print(tr.Side)
    print("Our delta", action)
    print("Current mid", tr.MidPrice)
    print("Our action", our_q)
    print("Comp Action", tr.QuotedPrice)
    print("Market step", tr.NextMidPrice)
    print("Status", tr.Traded)
    print("Expected gain", best_action[1][0])
    print("Actual gain", profit)
    print("")
    if tr.Traded == "DONE":
        profits.append(profit)
    else:
        profits.append(0)

Bid
Our delta 0.09
Current mid 108.199999999999
Our action 108.10999999999899
Comp Action 107.889999999999
Market step 108.319999999999
Status MISSED
Expected gain 0.09
Actual gain 0.21000000000000796

Bid
Our delta 0.03
Current mid 84.790000000001
Our action 84.760000000001
Comp Action 84.760000000001
Market step 84.740000000001
Status DONE
Expected gain 0.03
Actual gain -0.01999999999999602

Bid
Our delta 0.03
Current mid 84.740000000001
Our action 84.710000000001
Comp Action 84.730000000001
Market step 84.6100000000011
Status DONE
Expected gain 0.03
Actual gain 0

Bid
Our delta 0.09
Current mid 108.809999999999
Our action 108.71999999999899
Comp Action 108.759999999999
Market step 108.489999999999
Status DONE
Expected gain 0.09
Actual gain 0

Offer
Our delta 0.0
Current mid 84.600000000001
Our action 84.600000000001
Comp Action 84.6400000000011
Market step 84.6100000000011
Status DONE
Expected gain 0.0
Actual gain -0.010000000000090381

Bid
Our delta 0.0
Current mid 108.829999999999

Our action 109.939999999999
Comp Action 109.889999999999
Market step 110.029999999999
Status MISSED
Expected gain 0.09
Actual gain 0.09000000000000341

Bid
Our delta 0.03
Current mid 83.9900000000011
Our action 83.9600000000011
Comp Action 83.9700000000011
Market step 84.0300000000011
Status DONE
Expected gain 0.03
Actual gain 0

Offer
Our delta 0.1
Current mid 110.139999999999
Our action 110.239999999999
Comp Action 110.239999999999
Market step 110.539999999999
Status MISSED
Expected gain 0.1
Actual gain -0.29999999999999716

Bid
Our delta 0.1
Current mid 123.459999999999
Our action 123.359999999999
Comp Action 123.269999999999
Market step 123.279999999999
Status MISSED
Expected gain 0.1
Actual gain -0.0799999999999983

Offer
Our delta 0.14
Current mid 83.9700000000011
Our action 84.1100000000011
Comp Action 84.0100000000011
Market step 84.0600000000011
Status DONE
Expected gain 0.14
Actual gain 0

Bid
Our delta 0.09
Current mid 110.309999999999
Our action 110.21999999999899
Comp Acti

Bid
Our delta 0.09
Current mid 111.089999999999
Our action 110.99999999999899
Comp Action 111.009999999999
Market step 111.069999999999
Status DONE
Expected gain 0.09
Actual gain 0

Offer
Our delta 0.03
Current mid 83.5000000000012
Our action 83.5300000000012
Comp Action 83.6300000000011
Market step 83.4700000000011
Status MISSED
Expected gain 0.03
Actual gain 0.06000000000010175

Bid
Our delta 0.09
Current mid 111.099999999999
Our action 111.009999999999
Comp Action 110.809999999999
Market step 111.019999999999
Status MISSED
Expected gain 0.09
Actual gain 0.010000000000005116

Bid
Our delta 0.03
Current mid 83.4100000000011
Our action 83.3800000000011
Comp Action 83.3600000000012
Market step 83.4000000000011
Status DONE
Expected gain 0.03
Actual gain 0.01999999999999602

Offer
Our delta 0.0
Current mid 110.909999999999
Our action 110.909999999999
Comp Action 110.939999999999
Market step 110.949999999999
Status DONE
Expected gain 0.0
Actual gain -0.03999999999999204

Offer
Our delta 0.

Bid
Our delta 0.04
Current mid 83.0600000000011
Our action 83.02000000000109
Comp Action 83.0300000000011
Market step 83.1100000000011
Status DONE
Expected gain 0.04
Actual gain 0

Offer
Our delta 0.14
Current mid 83.1100000000011
Our action 83.2500000000011
Comp Action 83.1500000000011
Market step 83.1700000000011
Status DONE
Expected gain 0.14
Actual gain 0

Offer
Our delta 0.1
Current mid 111.239999999999
Our action 111.339999999999
Comp Action 111.269999999999
Market step 111.249999999999
Status DONE
Expected gain 0.1
Actual gain 0

Bid
Our delta 0.09
Current mid 111.249999999999
Our action 111.159999999999
Comp Action 111.109999999999
Market step 111.249999999999
Status MISSED
Expected gain 0.09
Actual gain 0.09000000000000341

Offer
Our delta 0.14
Current mid 126.959999999999
Our action 127.099999999999
Comp Action 127.019999999999
Market step 126.939999999999
Status DONE
Expected gain 0.14
Actual gain 0

Offer
Our delta 0.0
Current mid 111.189999999999
Our action 111.18999999999

Bid
Our delta 0.09
Current mid 111.329999999999
Our action 111.239999999999
Comp Action 111.299999999999
Market step 111.199999999999
Status DONE
Expected gain 0.09
Actual gain 0

Offer
Our delta 0.1
Current mid 111.199999999999
Our action 111.29999999999899
Comp Action 111.209999999999
Market step 111.019999999999
Status DONE
Expected gain 0.1
Actual gain 0

Offer
Our delta 0.01
Current mid 84.2100000000011
Our action 84.22000000000111
Comp Action 84.4400000000011
Market step 84.1900000000011
Status MISSED
Expected gain 0.01
Actual gain 0.030000000000001137

Bid
Our delta 0.04
Current mid 84.1900000000011
Our action 84.1500000000011
Comp Action 84.1300000000011
Market step 84.1600000000011
Status MISSED
Expected gain 0.04
Actual gain 0.010000000000005116

Bid
Our delta 0.04
Current mid 84.1600000000011
Our action 84.1200000000011
Comp Action 84.1300000000011
Market step 84.1600000000011
Status MISSED
Expected gain 0.04
Actual gain 0

Bid
Our delta 0.09
Current mid 110.799999999999
Our

...

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan


nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan


nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan


nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan


nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan
Our action 126.999999999999
Comp Action nan
Market step nan
Status nan
Expected gain 0.0
Actual gain 0

nan
Our delta 0.0
Current mid nan


In [457]:
sum(profits)

0.4200000000003854

In [458]:
profits.count(0)

5835

In [459]:
len(profits)

5989